Some fun experiments comparing the performance of using analytical solution vs gradient descent to solve linear regression.

In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error as mse

randn = np.random.randn
l2norm = np.linalg.norm
inv = np.linalg.inv

import torch.utils.benchmark as torchbench

def bench(fn):
    t0 = torchbench.Timer(
    stmt='fn()',
    globals={'fn': fn})
    return t0.timeit(5).mean * 1000

Create the design matrix X and the label y arrays by a linear transformation plus some normally distributed error.

In [7]:
nfeatures = 1024 + 1        # n
nsamples = nfeatures * 15   # m

b = randn(nfeatures)
print('(true) feature vector:\n', b)
X = randn(nsamples, nfeatures)
X[:,0] = 1
print('design matrix:\n', X)

(true) feature vector:
 [ 1.47043091 -0.69113223  0.45764818 ...  0.83138272 -0.6473143
  1.75372123]
design matrix:
 [[ 1.          0.22026668 -0.6724195  ... -0.79868081 -0.07222758
  -1.17574919]
 [ 1.         -0.28478734  0.1297956  ...  0.12191959  0.01505071
  -0.48263082]
 [ 1.         -2.37904183  0.80408426 ... -0.85769672  0.3418519
  -1.42630717]
 ...
 [ 1.          0.44553891 -1.48380682 ...  0.68697295  0.02512675
  -2.07690947]
 [ 1.          1.55324169 -1.46649164 ... -0.63692279 -0.54866655
  -0.47249879]
 [ 1.         -0.23864092  0.29223827 ...  1.0509838  -1.65980791
  -0.32913919]]


In [8]:
y_truth = X @ b
y = y_truth + randn(nsamples)

def direct_solution():
    # Complexity analysis
    # X.T @ X: X is m x n, so O(mn^2)
    # inv: O(n^3)
    # inv(...) @ X.T: O(mn^2)
    # last one is MV product: O(mn)
    b_hat = inv(X.T @ X) @ X.T @ y
    return b_hat

b_hat = direct_solution()
print('b hat:  ', b_hat)
print('b truth:', b)
print(f'MSE: {mse(b, b_hat):.8f}')
print(f'time: {bench(direct_solution):.3f} ms')
print(f'time of matmul: {bench(lambda: X.T @ X):.3f}')
t = X.T @ X
print(f'time of inverting: {bench(lambda: inv(t)):.3f}')

b hat:   [ 1.47990112 -0.69498175  0.45607926 ...  0.83427482 -0.64725524
  1.76541115]
b truth: [ 1.47043091 -0.69113223  0.45764818 ...  0.83138272 -0.6473143
  1.75372123]
MSE: 0.00007275
time: 960.215 ms
time of matmul: 403.077
time of inverting: 68.085


In [9]:
def gd_fixed_step(step=0.2, iters=1000):
    b = randn(nfeatures)
    for i in range(iters):
        grad = -2 * (y - X @ b) @ X / nsamples
        b -= step * grad
        ng = l2norm(grad)
        #print(ng)
        if ng < 1e-4:
            print(i)
            break
    return b
    
b_hat = gd_fixed_step()
print('b hat:  ', b_hat)
print('b truth:', b)
print(f'MSE: {mse(b, b_hat):.8f}')
print(f'time: {bench(gd_fixed_step):.3f} ms')

44
b hat:   [ 1.47989959 -0.69498235  0.45607675 ...  0.83427476 -0.64725468
  1.76541284]
b truth: [ 1.47043091 -0.69113223  0.45764818 ...  0.83138272 -0.6473143
  1.75372123]
MSE: 0.00007275
44
44
44
45
44
44
44
time: 289.730 ms


In [10]:
def gd_varied_step(init_step=0.2, iters=1000):
    step = init_step
    b = randn(nfeatures)
    error = (y - X @ b)
    prev_sse = np.sum(np.power(error, 2))
    for i in range(iters):
        error = (y - X @ b)
        sse = np.sum(np.power(error, 2))
        if sse < prev_sse:
            step *= 1.5
        else:
            step = init_step
        prev_sse = sse    
        grad = -2 * error @ X / nsamples
        b -= step * grad
        ng = l2norm(grad)
        if ng < 1e-4:
            print(i)
            break
    return b
    
b_hat = gd_varied_step()
print('b hat:  ', b_hat)
print('b truth:', b)
print(f'MSE: {mse(b, b_hat):.8f}')
print(f'time: {bench(gd_varied_step):.3f} ms')

17
b hat:   [ 1.47990109 -0.6949814   0.4560792  ...  0.83427472 -0.64725517
  1.76541146]
b truth: [ 1.47043091 -0.69113223  0.45764818 ...  0.83138272 -0.6473143
  1.75372123]
MSE: 0.00007275
17
17
17
17
17
17
17
time: 118.217 ms
